<a href="https://colab.research.google.com/github/PikasXYZ/Audio-Transcription-Translation-Summarization/blob/main/Audio_Transcription_%7C_Translation_%7C_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installation & imports

In [1]:
#@title Installation
!pip install pydub -q
!pip install yt-dlp -q
!pip install openai -q -U
!pip install pytube -q
!pip install python-docx -q
!pip install SpeechRecognition -q
!pip install tiktoken -q -U
!pip install google-cloud-translate
!pip install git+https://github.com/openai/whisper.git -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
#@title imports
# Standard libraries
import os
import re
import time
import shutil
import requests

# Third-party libraries
from openai import OpenAI
import yt_dlp
import whisper
import requests
import tiktoken
from docx import Document
from pytube import YouTube
from bs4 import BeautifulSoup
from pydub import AudioSegment
from pydub.silence import split_on_silence
import speech_recognition as sr
import ipywidgets as widgets

# Google libraries
from google.cloud import translate
import google.colab
from google.colab import userdata
from google.colab import drive, files

# IPython display
from IPython.display import clear_output

#Transcription Functions

In [6]:
#@title Transcribe Audio to Text by Open Source Whisper
def opensrc_whisper_transcript2text(audio, print_out, model):
  # ref: https://github.com/openai/whisper/blob/main
  while "This loop is to avoid sound chunk being too long":
    try:
      chunks = split_on_silence(
                    audio,
                    min_silence_len = silence, #experiment with this value for your target audio file
                    silence_thresh = sound.dBFS-14, #adjust this per requirement
                    keep_silence = silence #keep the silence for 0.5 second, adjustable as well
                    )

      if print_out: print(f"【Transcription】 (silence={silence}ms)")

      transcriptor = whisper.load_model(model)

      output = ''
      for i, audio_chunk in enumerate(chunks, start=1):
        audio_chunk.export('chunk.wav', format="wav")
        try:
          result = transcriptor.transcribe('chunk.wav')["text"]
          transcript = break_lines(transcript)

        except sr.UnknownValueError as e:
          print(type(e).__name__,':', str(e))

        else:
          if print_out: print(transcript)
          output += transcript+'\n'

    except:
      if silence>=100: silence -= 50
      else: raise Exception("Cannot deal with this audio, please try another one")
      #print(f"Sound chunks are too long! Trying to decrease SILENCE to {silence}ms.")
      clear_output(wait=True)

    else:
      break

  return output

In [7]:
#@title Transcribe Audio to Text by OpenAI Whisper (Faster than the above merely)
def OpenAI_whisper_transcript2text(audio, print_out):
  #ref: https://platform.openai.com/docs/guides/speech-to-text/quickstart
  global openai_client
  silence=600
  while "This loop is to avoid sound chunk being too long":
    try:
      chunks = split_on_silence(
                    audio,
                    min_silence_len = silence, #experiment with this value for your target audio file
                    silence_thresh = sound.dBFS-14, #adjust this per requirement
                    keep_silence = silence #keep the silence for 0.5 second, adjustable as well
                    )

      if print_out: print(f"【Transcription】 (silence={silence}ms)")

      output = ''
      for i, audio_chunk in enumerate(chunks, start=1):
        audio_chunk.export('chunk.wav', format="wav")
        audio_file = open('chunk.wav', "rb")
        try:
          transcript = openai_client.audio.transcriptions.create(model="whisper-1", file=audio_file).text
          transcript = break_lines(transcript)

        except sr.UnknownValueError as e:
          print(type(e).__name__,':', str(e))
        else:
          if print_out: print(transcript)
          output += transcript+'\n'

    except:
      if silence>=100: silence -= 50
      else: raise Exception("Cannot deal with this audio, please try another one")
      #print(f"Sound chunks are too long! Trying to decrease SILENCE to {silence}ms. ")
      clear_output(wait=True)

    else:
      break

  return output

#Translation Functions

In [32]:
#@title OpenAI ChatGPT Translate
def OpenAI_ChatGPT_translate(texts, translaion_language, model, print_out):
  global openai_client

  if type(texts) == str: texts = [texts]

  output = ""
  for text in texts:
    messages = [{"role":"system", "content": f"You are a helpful translator. Please help me to translate following text and output in {translaion_language}."}]
    messages.append({"role": "user", "content": text})
    response = openai_client.chat.completions.create(model=model, messages=messages)
    translate = response.choices[0].message.content
    translate = break_lines(translate)
    if print_out: print(translate)
    output += translate

  return output

In [21]:
#@title Google Translate
def Google_translate(texts: list ,target_language: str, print_out=True) -> str:
  texts = [texts] if type(texts) == str else texts

  global GOOGLE_TRANSLATE_API_KEY, google_translation_support_languages

  output = ""
  for text in texts:
    # ref: https://ithelp.ithome.com.tw/m/articles/10218704

    #detect
    google_detection_api_url = f"https://translation.googleapis.com/language/translate/v2/detect/?q={text}&key={GOOGLE_TRANSLATE_API_KEY}"
    response = requests.get(google_detection_api_url)
    source_language = response.json().get('data').get('detections')[0][0].get('language')

    #check target_language
    if target_language not in google_translation_support_languages:
      print(f"\"{target_language}\" isn't supported by Google Translate, please try the following:")
      print('\n'.join([key for key, item in google_translation_support_languages.items()]))
      raise Exception("translaion_language Error!")

    #translate
    google_translation_api_url = f"https://translation.googleapis.com/language/translate/v2/?q={text}&source={source_language}&target={google_translation_support_languages[target_language]}&key={GOOGLE_TRANSLATE_API_KEY}"
    response = requests.get(google_translation_api_url)
    try:
      translate = break_lines(response.json().get('data').get('translations')[0].get('translatedText'))
      if print_out: print(translate)
      output += translate
    except Exception as e:
      print(f"Request failed with status code {response.status_code}")
      print(type(e).__name__)

  return output

#Summarization Functions

In [30]:
#@title Summarize w/ OpenAI ChatGPT
def summarize(texts, language, model, print_out):
  global openai_client

  if type(texts) == str: texts = [texts]

  output = ''
  for text in texts:
    messages = [{"role":"system", "content": f"Please summarize following audio transcription content briefly and output in {language}."}]
    messages.append({"role": "user", "content": text})
    response = openai_client.chat.completions.create(model=model, messages=messages)
    reply = response.choices[0].message.content
    reply = break_lines(reply)
    if print_out: print(reply)
    output += reply

  return output

#Other Functions

In [3]:
#@title Break Lines
def break_lines(text: str) -> str:
    pattern = r'([。？！.?!])'
    lines = [line.strip() for line in re.split(pattern, text) if line]

    result = []
    for i in range(0, len(lines), 2):
      lines[i] += lines[i+1] if i+1 < len(lines) else ''
      result.append(lines[i])

    return '\n'.join(result)
print(break_lines("Hello! How are you?"))

Hello!
How are you?


In [5]:
#@title Split the Text to Prevent it Containing Too More Token
# encoding = tiktoken.get_encoding("cl100k_base")
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
def split_by_token(text, spliter = '\n', max_token = 2999):
  #Initialize
  text_chunks = text.split(spliter)
  texts = []

  #Split
  output = ''
  for chunk in text_chunks:
    if len(encoding.encode(output + chunk)) >= max_token:
      texts.append(output)
      output = chunk + spliter
    else:
      output += chunk + spliter
  else:
    texts.append(output)

  return texts

In [10]:
#@title Output as docx
def save_as_docx(text, filename):
  if os.path.exists(filename): os.remove(filename)
  document = Document()
  document.add_paragraph(text)
  document.save(filename)

In [11]:
#@title Output as txt
def save_as_txt(text, filename):
  if os.path.exists(filename): os.remove(filename)
  document = open(filename, "w")
  document.write(text)
  document.close

In [12]:
#@title Get Userdata Safely
def get_userdata_safely(key):
  #What's userdata.get? https://drlee.io/how-to-use-secrets-in-google-colab-for-api-key-protection-a-guide-for-openai-huggingface-and-c1ec9e1277e0
  try:
    data = userdata.get(key)
    print(f"Userdata found! ({key = })")
  except:
    data = ""
    print(f"Userdata not found! ({key = })")
  return data

#Preprocessing

In [13]:
#@title settings
GOOGLE_TRANSLATE_API_KEY = get_userdata_safely('GOOGLE_TRANSLATE_API_KEY')
GEMINI_API_KEY = get_userdata_safely('GEMINI_API_KEY')
OPENAI_API_KEY = get_userdata_safely('OPENAI_API_KEY')
openai_client = OpenAI(api_key=OPENAI_API_KEY)

Userdata found! (key = 'GOOGLE_TRANSLATE_API_KEY')
Userdata found! (key = 'GEMINI_API_KEY')
Userdata found! (key = 'OPENAI_API_KEY')


In [19]:
#@title get Google translation support languages
if GOOGLE_TRANSLATE_API_KEY:
  google_translation_support_languages_api_url = f"https://translation.googleapis.com/language/translate/v2/languages/?target=en&key={GOOGLE_TRANSLATE_API_KEY}"
  response = requests.get(google_translation_support_languages_api_url)
  google_translation_support_languages = {x['name']:x['language'] for x in response.json().get('data').get('languages')}
  print("【Google translation support languages】")
  for k,v in google_translation_support_languages.items():
    print(f"{k} ({v})")

【Google translation support languages】
Afrikaans (af)
Albanian (sq)
Amharic (am)
Arabic (ar)
Armenian (hy)
Assamese (as)
Aymara (ay)
Azerbaijani (az)
Bambara (bm)
Basque (eu)
Belarusian (be)
Bengali (bn)
Bhojpuri (bho)
Bosnian (bs)
Bulgarian (bg)
Catalan (ca)
Cebuano (ceb)
Chichewa (ny)
Chinese (Simplified) (zh-CN)
Chinese (Traditional) (zh-TW)
Corsican (co)
Croatian (hr)
Czech (cs)
Danish (da)
Divehi (dv)
Dogri (doi)
Dutch (nl)
English (en)
Esperanto (eo)
Estonian (et)
Ewe (ee)
Filipino (tl)
Finnish (fi)
French (fr)
Frisian (fy)
Galician (gl)
Ganda (lg)
Georgian (ka)
German (de)
Greek (el)
Guarani (gn)
Gujarati (gu)
Haitian Creole (ht)
Hausa (ha)
Hawaiian (haw)
Hebrew (he)
Hindi (hi)
Hmong (hmn)
Hungarian (hu)
Icelandic (is)
Igbo (ig)
Iloko (ilo)
Indonesian (id)
Irish Gaelic (ga)
Italian (it)
Japanese (ja)
Javanese (jv)
Kannada (kn)
Kazakh (kk)
Khmer (km)
Kinyarwanda (rw)
Konkani (gom)
Korean (ko)
Krio (kri)
Kurdish (Kurmanji) (ku)
Kurdish (Sorani) (ckb)
Kyrgyz (ky)
Lao (lo)
Latin (la

In [15]:
#@title Settings of OpenAI (Ignorable if not going to use relative tools)
#@markdown ####What model to use?<br/>
GPT_model = "gpt-3.5-turbo" #@param ["gpt-4o","gpt-4-turbo","gpt-3.5-turbo"]

#Main
###⚠️After changing any parameter in a cell, please run that cell again.⚠️
####⚠️After changing any parameter in a cell, please run that cell again.⚠️
#####⚠️After changing any parameter in a cell, please run that cell again.⚠️

In [16]:
#@title Download Video(s) or Audio(s)
#@markdown ### Choose the source of video or audio:
#@markdown ##### ㅤ
source = "share link of Google Drive" #@param ["Youtube URL","file path of Google Drive","folder path of Google Drive","share link of Google Drive", "upload from my computer"]
#@markdown ##### ㅤ
#@markdown ### Set path/URL according to the source you choose:
#@markdown ##### ㅤ
youtube_url = "https://www.youtube.com/watch?v=dQw4w9WgXcQ" #@param {type:"string"}
file_path_of_Google_Drive = "/content/drive/MyDrive/Rick Astley - Never Gonna Give You Up (Official Music Video).mp3" #@param {type:"string"}
folder_path_of_Google_Drive = "/content/drive/MyDrive/" #@param {type:"string"}
share_link_of_Google_Drive = "https://drive.google.com/file/d/1RxvmTPfHzaVBmSH99Ds6x7R4xecFmABz/view?usp=drive_link" #@param {type:"string"}
#@markdown ##### ㅤ
#@markdown ### Save mp3:
#@markdown ##### ㅤ
YouTube_mp3_to_my_computer = False #@param {type:"boolean"}
YouTube_mp3_to_Google_Drive = False #@param {type:"boolean"}
output_folder_of_Google_Drive = "/content/drive/MyDrive" #@param {type:"string"}

if YouTube_mp3_to_Google_Drive:
  drive.mount('/content/drive')
  if not os.path.isdir(output_folder_of_Google_Drive):
    os.mkdir(output_folder_of_Google_Drive)

match source:
  case _ if source.startswith("Youtube URL"):
    #set YouTube download options
    ydl_opts = {
          'format': 'bestaudio/best',
          'outtmpl': "%(title)s.%(ext)s",
          'postprocessors': [{
                    'key': 'FFmpegExtractAudio',
                    'preferredcodec': 'mp3', #download type
                    'preferredquality': '192',
                    }],
          'ignoreerrors': True #Ignore private videos
          }

    sounds = []
    filenames = []
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
      info_dict = ydl.extract_info(youtube_url, download = True) #Extract info. from YT URL
      audios = info_dict['entries'] if 'entries' in info_dict else [info_dict] #'entries' appears in info_dict when multiple videos are downloaded.
      for audio in audios:
        try:
          filename = os.path.splitext(ydl.prepare_filename(audio))[0]
          filenames.append(filename)
          sounds.append(AudioSegment.from_mp3(filename+'.mp3'))
          if YouTube_mp3_to_Google_Drive:
            shutil.copy2(filename+'.mp3', output_folder_of_Google_Drive)
          if YouTube_mp3_to_my_computer:
            google.colab.files.download(filename+'.mp3')
        except Exception as e:
          print(type(e).__name__," : ",str(e))


    def on_dl_button_clicked(button):
      for audio in audios:
        try: filename = os.path.splitext(ydl.prepare_filename(audio))[0]
        except: print("Found an invalid video.")
        else: google.colab.files.download(filename+'.mp3')

    dl_buttton = widgets.Button(description="Download Again")
    dl_buttton.on_click(on_dl_button_clicked)
    if YouTube_mp3_to_my_computer:
      display(dl_buttton) #Unknown error usually occurs while lots of videos are downloading.

  case "file path of Google Drive":
    drive.mount('/content/drive')
    filenames = [os.path.splitext(os.path.basename(file_path_of_Google_Drive))[0]]
    sounds = [AudioSegment.from_file(file_path_of_Google_Drive)]

  case "folder path of Google Drive":
    drive.mount('/content/drive')
    filenames = []
    sounds = []
    for root, dirs, files in os.walk(folder_path_of_Google_Drive):
      for name in files:
        if name.endswith(('.mp4', '.mov', '.avi', '.mkv', '.mp3', '.wav', '.flac')):
          filenames.append(name.split('.')[0])
          sounds.append(AudioSegment.from_file(os.path.join(root, name)))

  case "share link of Google Drive":
    assert share_link_of_Google_Drive.startswith("https://drive.google.com/file/d/"), "Invalid Google Drive link format"
    file_id = share_link_of_Google_Drive.split("https://drive.google.com/file/d/")[-1].split("/")[0]

    try:
      response = requests.get(share_link_of_Google_Drive)
      soup = BeautifulSoup(response.text, 'html.parser')
      filenames = [soup.find('title').text.strip().split(" - Google Drive")[0]]
    except:
      filenames = ["download"]

    filename = filenames[0]
    !gdown --id $file_id --output "$filename"
    sounds = [AudioSegment.from_file(filenames[0])]

  case "upload from my computer":
    filenames = []
    sounds = []
    upload = google.colab.files.upload()
    for k in upload.keys():
      if k.endswith(('.mp4', '.mov', '.avi', '.mkv', '.mp3', '.wav', '.flac')):
        filenames.append(k.split('.')[0])
        sounds.append(AudioSegment.from_file(k))

  case _:
    raise Exception("Source Error!")

#print out content unless there're multiple files to process
print_out_content = len(sounds)==1

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1RxvmTPfHzaVBmSH99Ds6x7R4xecFmABz
To: /content/Never Gonna Give You Up.mp3
100% 3.39M/3.39M [00:00<00:00, 76.9MB/s]


In [17]:
#@title Transcription
#@markdown ####Settings:ㅤ
transcription_tool = "OpenAI-whisper" #@param ["OpenAI-whisper", "whisper-tiny", "whisper-base", "whisper-small", "whisper-medium", "whisper-large", "whisper-tiny.en", "whisper-base.en", "whisper-small.en", "whisper-medium.en"]
output_type = "txt" #@param ["txt","docx"]
output_to_my_computer = False #@param {type:"boolean"}
output_to_Google_Drive = False #@param {type:"boolean"}
output_folder_of_Google_Drive = "/content/drive/MyDrive" #@param {type:"string"}
if output_to_Google_Drive:
  drive.mount('/content/drive')
  if not os.path.isdir(output_folder_of_Google_Drive):
    os.mkdir(output_folder_of_Google_Drive)

transcription_output_chunks_list = []
for sound, filename in zip(sounds, filenames):

  #transcription process
  match transcription_tool:
    case "OpenAI-whisper":
        transcription_output = OpenAI_whisper_transcript2text(sound,
                                    print_out_content)
    case _ if "whisper-" in transcription_tool:
        transcription_output = opensrc_whisper_transcript2text(sound,
                                     print_out_content,
                                     transcription_tool.split('-')[-1])
  if not print_out_content:
    print(filename, " is transcripted.")
  transcription_output_chunks = split_by_token(transcription_output)
  transcription_output_chunks_list.append(transcription_output_chunks)

  #transcription output
  output_name = f"{filename}_transcripted_by_{transcription_tool}.{output_type}"
  match output_type:
    case "txt":
      save_as_txt(transcription_output, output_name)
    case "docx":
      save_as_docx(transcription_output, output_name)

  #save files
  if output_to_Google_Drive:
    shutil.copy2(output_name, output_folder_of_Google_Drive)
  if output_to_my_computer:
    google.colab.files.download(output_name)

【Transcription】 (silence=150ms)
BOOM
We're no strangers to love You know the rules, and so do I I feel commitments while I'm thinking of You wouldn't get this from any other guy I just wanna tell you how I'm feeling Gotta make you understand Never gonna give you up Never gonna let you down Never gonna run around and desert you Never gonna make you cry Never gonna say goodbye Never gonna tell a lie and hurt you We've known each other for so long Your heart's been aching but you're too shy to say it Inside we both know what's been going on We know the game and we're gonna play it And if you ask me how I'm feeling Don't tell me you're too blind to see Never gonna give you up Never gonna let you down Never gonna run around and desert you Never gonna make you cry Never gonna say goodbye Never gonna tell a lie and hurt you Never gonna give you up Never gonna let you down Never gonna run around and desert you Never gonna make you cry Never gonna say goodbye Never gonna tell a lie and hurt you

In [33]:
#@title Translation (⚠️Transcription must be done before)
#@markdown ####Settings:
translation_tool = "OpenAI-ChatGPT" #@param ["OpenAI-ChatGPT","Google-Translate"]
language = "Chinese (Traditional)" #@param {type: "string"}
output_type = "txt" #@param ["txt","docx"]
output_to_my_computer = False #@param {type:"boolean"}
output_to_Google_Drive = False #@param {type:"boolean"}
output_folder_of_Google_Drive = "/content/drive/MyDrive" #@param {type:"string"}
if output_to_Google_Drive:
  drive.mount('/content/drive')
  if not os.path.isdir(output_folder_of_Google_Drive):
    os.mkdir(output_folder_of_Google_Drive)

for transcription_output_chunks, filename in zip(transcription_output_chunks_list, filenames):

  #translation process
  output = "Error! "*10+'\n'
  match translation_tool:
    case "Google-Translate":
        output = Google_translate(transcription_output_chunks,
                      language,
                      print_out_content)
    case "OpenAI-ChatGPT":
        output = OpenAI_ChatGPT_translate(transcription_output_chunks,
                          language,
                          GPT_model,
                          print_out_content)

  if not print_out_content:
    print(filename, " is translated.")

  #translation output
  output_name = f"{filename}_translated_by_{translation_tool}.{output_type}"
  match output_type:
    case "txt":
        save_as_txt(output, output_name)
    case "docx":
        save_as_docx(output, output_name)

  #save files
  if output_to_Google_Drive:
    shutil.copy2(output_name, output_folder_of_Google_Drive)
  if output_to_my_computer:
    google.colab.files.download(output_name)


BOOM
我們並不陌生 我們都知道規矩 我感到投入當我想起 你不會從其他人那裡得到這些 我只想告訴你我的感受 必須讓你明白 永遠不會放棄你 永遠不會讓你失望 永遠不會四處亂跑並拋棄你 永遠不會讓你哭泣 永遠不會說再見 永遠不會說謊傷害你 我們彼此相識已久 你的心一直在痛苦但你太害羞說出口 我們內心明白發生了什麼 我們知道這場遊戲並將一起玩 要是你問我感受 如何 別告訴我你太盲目看不見 永遠不會放棄你 永遠不會讓你失望 永遠不會四處亂跑並拋棄你 永遠不會讓你哭泣 永遠不會說再見 永遠不會說謊傷害你 永遠不會放棄你 永遠不會讓你失望 永遠不會四處亂跑並拋棄你 永遠不會讓你哭泣 永遠不會說再見 永遠不會說謊傷害你 永遠不會放棄你 永遠不會讓你失望 永遠不會四處亂跑並拋棄你 ♪ 洗掉你，我將憎惡你 ♪ 謝謝你。
好的。
♪ 喔，說。
♪


In [31]:
#@title Summarization (⚠️Transcription must be done before)
#@markdown ####Settings:
output_type = "txt" #@param ["txt","docx"]
language = "Chinese (Traditional)" #@param {type: "string"}
output_to_my_computer = False #@param {type:"boolean"}
output_to_Google_Drive = False #@param {type:"boolean"}
output_folder_of_Google_Drive = "/content/drive/MyDrive" #@param {type:"string"}
if output_to_Google_Drive:
  drive.mount('/content/drive')
  if not os.path.isdir(output_folder_of_Google_Drive):
    os.mkdir(output_folder_of_Google_Drive)

for transcription_output_chunks, filename in zip(transcription_output_chunks_list, filenames):

  #summarization output
  output = summarize(transcription_output_chunks,
            language,
            GPT_model,
            print_out_content)

  if not print_out_content: print(filename, " is summarized.")

  #summarization output
  output_name = f"{filename}_summarized.{output_type}"
  match output_type:
    case "txt":
        save_as_txt(output, output_name)
    case "docx":
        save_as_docx(output, output_name)

  #save files
  if output_to_Google_Drive:
    shutil.copy2(output_name, output_folder_of_Google_Drive)
  if output_to_my_computer:
    google.colab.files.download(output_name)

這段內容是一首歌曲的歌詞，描述了一個人對另一個人的承諾和愛意。
